### Introduction to functools 

Python comes with a fun module called **functools**. The functions inside functools are considered "higher-order" functions which can act on or return other functions. We will look at following packages of functool functions :

+ lru_cache
+ partials
+ singledispatch
+ wraps

https://docs.python.org/3/library/functools.html

### Caching with functools.lru_cache

The functools module provides a handy decorator called **lru_cache**. Note that it was added in Python 3.2. According ot the documentation, it will "wrap a function with a memorizing callable that saves up to the maxsize most recent calls". In other words, it's a decorator that adds caching to the function it decorates. 

Let's write a quick example that will grab various web page from Python documentation site.


In [11]:
import urllib.error
import urllib.request

from functools import lru_cache

@lru_cache(maxsize = 24)
def get_webpage(module):
    """
    Gets the specified Python module web page
    """
    webpage = "https://docs.python.org/3/library/{}.html".format(module)
    try:
        with urllib.request.urlopen(webpage) as request:
            val = request.read()
            #print(val)
            return val
    except urllib.error.HTTPError:
        return none
    
if __name__ == "__main__":
    modules = ['functools', 'collections', 'os', 'sys']
    for module in modules:
        page = get_webpage(module)
    if page:
        print("{} module page found".format(module))
    
    

sys module page found


In the code above, we decorate our **get_webpage** function with **lru_cache** and set its max size to 24 calls. Then we set up a webpage string variable and pass in which module we want our function to fetch. I found that this works best if you run it in a Python interpreter, such as IDLE. This allows you to run the loop a couple of times against the function. What you will quickly see is that the first time it runs the code, the output is printed our relatively slowly. But if you run it again in the same session, you will see that it prints immediately which demonstrates that the lru_cache has cached the calls correctly. Give this a try in your own interpreter instance to see the results for yourself.

There is also **typed** parameter that we can pass to the decorator. It is a Boolean that tells the decorator to cache arguments of different types separately if types is set to **True**

In [17]:
from functools import lru_cache

@lru_cache(maxsize = 3)
def square(x :float) -> float:
    print(f'running: {x}')
    return x * x

print(square(10))
print(square(10))
print(square(10))
print(square(2))
print(square(3))
print(square(4))
print(square(2))
print(square(3))
print(square(10))  # forgotten 10 as maxsize is 3 

# It will not cache 
print(square.__wrapped__(10))
print(square.__wrapped__(10))
print(square.__wrapped__(10))

running: 10
100
100
100
running: 2
4
running: 3
9
running: 4
16
4
9
running: 10
100
running: 10
100
running: 10
100
running: 10
100


In [18]:
## something like singleton

@lru_cache(maxsize =1)
def get_thing() -> int:
    # pretend this is expensive 
    print("expensive")
    return 42

print(get_thing())
print(get_thing())
print(get_thing())


expensive
42
42
42


### functool.partial 

One of the functool classes is the **partial** class. You can use it to create new function with partial application of the arguments  and keywords that you pass to it. You can use partial to *freeze*  a portion of your function's arguments and/or keyword which results in a new object. Another way to put it is the partial creates a new function with some defaults.

In [19]:
from functools import partial

def add(x,y):
    return x+y

p_add = partial(add, 2)
print(p_add(4))

6


Here we create a simple adding function that returns the result of adding its arguments, x and y. Next we create a new callable by creating an instance of partial and passing it our function and an argument for that function. In other words, we are basically defaulting the x parameter of our add function to the number 2. Finally we call or new callable, p_add, with the argument of the number 3 which results in 6 because 2+4 = 6.

One handy use case for partials is to pass arguments to callbacks. :et's take a look using exPython:



In [3]:
# Conda install wxpython
import wx 

from functools import partial


class MainFrame(wx.Frame):
    """
    This app shows a group of buttons
    """

    def __init__(self, *args, **kwargs):
        """Constructor"""
        super(MainFrame, self).__init__(parent=None, title='Partial')
        panel = wx.Panel(self)

        sizer = wx.BoxSizer(wx.VERTICAL)
        btn_labels = ['one', 'two', 'three']
        for label in btn_labels:
            btn = wx.Button(panel, label=label)
            btn.Bind(wx.EVT_BUTTON, partial(self.onButton, label=label))
            sizer.Add(btn, 0, wx.ALL, 5)

        panel.SetSizer(sizer)
        self.Show()

    def onButton(self, event, label):
        """
        Event handler called when a button is pressed
        """
        print('You pressed: ' +  str(label))


if __name__ == '__main__':
    app = wx.App(False)
    frame = MainFrame()
    app.MainLoop()

You pressed: one
You pressed: two
You pressed: three
You pressed: one
You pressed: two
You pressed: two


Here we use partial to call the onButton event handler with an extra argument, which happens to be the button’s label. This might not seem all that useful to you, but if you do much GUI programming, you’ll see a lot of people asking how to do this sort of thing. Of course, you could also use a lambda instead for passing arguments to callbacks.

One use case that we’ve used at work was for our automated test framework. We test a UI with Python and we wanted to be able to pass a function along to dismiss certain dialogs. Basically you would pass a function along with the name of the dialog to dismiss, but it would need to be called at a certain point in the process to work correctly. Since I can’t show that code, here’s a really basic example of passing a partial function around:

In [4]:
from functools import partial

def add(x,y):
    """"""
    return x+y

def multiply(x,y):
    """"""
    return x*y

def run(func):
    """"""
    print(func())
    
def main():
    """"""
    a1 = partial(add,1,2)
    m1 = partial(multiply,2,3)
    run(a1)
    run(m1)
    
if __name__ == '__main__':
    main()

3
6


Here we create a couple of partial functions in our main function. Next we pass those partials to our run function, call it and then print out the result of the function that was called.

### Function Overloading with functools.singledispatch

Python fairly recently added partial support for function overloading in Python 3.4. They did this by adding a neat little decorator to the functools module called singledispatch. This decorator will transform your regular function into a single dispatch generic function. Note however that singledispatch only happens based on the first argument's type. 

In [10]:
from functools import singledispatch

@singledispatch
def add(a,b):
    raise NotImplementedError("Unsupported type")
    
@add.register(int)
def _(a,b):
    print("First argument type is: ", type(a))
    return a+b

@add.register(str)
def _(a,b):
    print("First argument type is: ", type(a))
    return (a+b)

@add.register(list)
def _(a,b):
    print("First argument type is: ", type(a))
    return (a+b)

if __name__ == '__main__':
    print(add(2,3))
    print(add("Hello","World"))
    print(add([1,2,3,4,5],[6,7,8,9]))
    print(add({1:1},{2:2}))



First argument type is:  <class 'int'>
5
First argument type is:  <class 'str'>
HelloWorld
First argument type is:  <class 'list'>
[1, 2, 3, 4, 5, 6, 7, 8, 9]


NotImplementedError: Unsupported type

Here we import singledispatch from functools and apply it to a simple function that we call add. This function is our catch-all function and will only get called if none of the other decorated functions handle the type passed. You will note that we currently handle integers,string and list as the first argument. If we were to call our add function with something else, such as a dictionary, then it would raise a NotImplemented Error.\

It calls the appropriate function based on the first argument's type. If the type isn't handled, then we raise a NotImplementedError. IF you want to know what types we are currently handling, you can try this:

In [12]:
print(add.registry.keys())

dict_keys([<class 'object'>, <class 'int'>, <class 'str'>, <class 'list'>])


This tells us that we can handle strings,integers, list and objects(the default). The singledispatch decorator also supports decorator stacking. This allows us to create an overloaded function that can handle multiple types.

In [15]:
from functools import singledispatch
from decimal import Decimal

@singledispatch
def add(a,b):
    raise NotImplementedError('Unsupported type')
    
@add.register(float)
@add.register(Decimal)
def _(a,b):
    print("First argument is of type ", type(a))
    print(a + b)
    
if __name__=='__main__':
    add(1.23, 5.5)
    add(Decimal(100.5), Decimal(10.789))
    
print(add.registry.keys())

First argument is of type  <class 'float'>
6.73
First argument is of type  <class 'decimal.Decimal'>
111.2889999999999997015720510
dict_keys([<class 'object'>, <class 'decimal.Decimal'>, <class 'float'>])


Because of the way all these functions were written, we could stack the decorators to handle all the cases in previous example. However, in normal overloaded case, each overload will call different code instead of doing the same thing.

### functools.wraps

There is a little known tool that I wanted to cover in this section. It is called wraps and it too is a part of the functools module. You can use wraps a a decorator to fix docstring and names of decorated functions. Why  does this matter? This sounds like a weird edge case at first, but if your're writing an API or any code that someone other than yourself will be using, then this could be important. The reason being that when you use Python's introspection to figure out someone else's code, a decorated function will return the wrong information.

In [17]:
def another_function(func):
    """
    A function that accepts another function
    """
    def wrapper():
        """
        A wrapping function
        """
        val = "The result of %s is %s" %(func(), eval(func()))
        return val
    return wrapper

@another_function
def a_function():
    """
    A pretty useless function
    """
    return "1+1"

if __name__ == '__main__':
    print(a_function.__name__)
    print(a_function.__doc__)

wrapper

        A wrapping function
        


In this code, we decorate the function called a_function with another_function. You can check a_function’s name and docstring by printing them out using the function’s __name__ and __doc__ properties.

How do we fix this little mess? The Python developers have given us the solution in functools.wraps!

In [18]:
from functools import wraps

def another_function(func):
    """ A function that accepts another function"""
    
    @wraps(func)
    def wrapper():
        """ a wrapping function"""
        val = "The result of %s is %s" %(func(), eval(func()))
        return val
    return wrapper

@another_function
def a_function():
    """A pretty useless function"""
    return "1+1"

if __name__ == '__main__':
    print(a_function.__name__)
    print(a_function.__doc__)

a_function
A pretty useless function


Here we import wraps from the functools module and use it as a decorator for the nested wrapper function inside of another_function. Now we have right name and docstring